# Chapter 4: Complete Example
In this chapter, we show two complete examples
1. comparing three decision making models using negative log-likelihood on simulated data
2. comparing three associative learning models using Bayesian Information Criterion on simulated data

## Imports

In [1]:
from ldmunit.models import decision_making, associative_learning
from ldmunit.models.utils import multi_from_single_interactive
from ldmunit.tests import NLLTest, BICTest

from os.path import join as pathjoin
from os import getcwd
import sciunit
sciunit.settings['CWD'] = getcwd()
DATA_PATH = pathjoin('..', 'data')

from read_example_data import get_simulation_data, get_model_params

## Negative Log-likelihood Tests

In [2]:
rwck = get_simulation_data(pathjoin(DATA_PATH, 'multi-rwck.csv'), 3)
nll_rwck = NLLTest(name='RWCK NLL', observation=rwck)

rr = get_simulation_data(pathjoin(DATA_PATH, 'multi-random-responding.csv'), 3)
nll_rr = NLLTest(name='RR NLL', observation=rr)

nwsls = get_simulation_data(pathjoin(DATA_PATH,'multi-nwsls.csv'), 3)
nll_nwsls = NLLTest(name='NWSLS NLL', observation=nwsls)

## Decision Making Models

In [3]:
MultiRWCKModel = multi_from_single_interactive(decision_making.RWCKModel)
MultiNWSLSModel = multi_from_single_interactive(decision_making.NWSLSModel)
MultiRandomRespondModel = multi_from_single_interactive(decision_making.RandomRespondModel)

n_action = 3
n_obs = 3

param_list = get_model_params(pathjoin(DATA_PATH, 'multi-rwck_prior.csv'))
multi_rwck = MultiRWCKModel(param_list, n_action=n_action, n_obs=n_obs)

param_list = get_model_params(pathjoin(DATA_PATH, 'multi-random-responding_prior.csv'))
multi_rr = MultiRandomRespondModel(param_list, n_action=n_action, n_obs=n_obs)

param_list = get_model_params(pathjoin(DATA_PATH,'multi-nwsls_prior.csv'))
multi_nwsls = MultiNWSLSModel(param_list, n_action=n_action, n_obs=n_obs)

## Run Test Suite

In [4]:
nll_suite = sciunit.TestSuite([nll_rwck, nll_rr, nll_nwsls], name="NLL suite")
nll_suite.judge([multi_rwck, multi_rr, multi_nwsls])

RWCK NLL RR NLL NWSLS NLL
RWCKModel               325    330       282
RandomRespondModel      381    301       372
NWSLSModel              474    521       189

## BIC Tests

In [5]:
krw_norm = get_simulation_data(pathjoin(DATA_PATH, 'multi-krw_norm.csv'), 3, True)
bic_krw_norm = BICTest(name="krw_norm BIC", observation=krw_norm)

lsspd = get_simulation_data(pathjoin(DATA_PATH, 'multi-lsspd.csv'), 3, True)
bic_lsspd = BICTest(name='lsspd BIC', observation=lsspd)

bb = get_simulation_data(pathjoin(DATA_PATH, 'multi-beta_binomial.csv'), 3, True)
bic_bb = BICTest(name='Beta Binomial BIC', observation=bb)

## Associative Learning Models

In [6]:
MultiKrwNormModel = multi_from_single_interactive(associative_learning.KrwNormModel)
MultiBetaBinomialModel = multi_from_single_interactive(associative_learning.BetaBinomialModel)
MultiLSSPDModel = multi_from_single_interactive(associative_learning.LSSPDModel)

param_list = get_model_params(pathjoin(DATA_PATH, 'multi-krw_norm_prior.csv'))
multi_krw_norm = MultiKrwNormModel(param_list, n_obs=4)

param_list = get_model_params(pathjoin(DATA_PATH, 'multi-lsspd_prior.csv'))
multi_lsspd = MultiLSSPDModel(param_list, n_obs=4)

param_list = get_model_params(pathjoin(DATA_PATH, 'multi-beta_binomial_prior.csv'))
multi_bb = MultiBetaBinomialModel(param_list, n_obs=4)

## Run Test Suite

In [7]:
bic_al_suite = sciunit.TestSuite([bic_krw_norm, bic_lsspd, bic_bb], name="BIC suite for associative learning")
bic_al_suite.judge([multi_krw_norm, multi_lsspd, multi_bb])

krw_norm BIC lsspd BIC Beta Binomial BIC
KrwNorm          2.37e+03  7.43e+16          3.86e+03
LSSPD            2.62e+03   1.3e+16          2.77e+03
BetaBinomial     2.12e+03     2e+16          2.09e+03